In [26]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [27]:
import torch
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 2080 Ti'

In [28]:
# import wandb

# # start a new wandb run to track this script
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="wordpiece-bert-project-epoch24",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 2e-5,
#     "weight_decay" : 0.01,
#     "architecture": "wordpiece-bert-base",
#     "dataset": "bdnews24",
#     "epochs": 24,
#     "batch size": 64,
#     }
# )


In [29]:
from datasets import load_dataset

prothom_alo_dataset = load_dataset("text", data_files="../datasets/Bangla Prothom Alo.txt", split="train")

Found cached dataset text (/home/ashahri1/.cache/huggingface/datasets/text/default-66e892579a4abe66/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2)


In [30]:
prothom_alo_dataset.set_format("pandas")

In [31]:
prothom_alo_df = prothom_alo_dataset[:]

In [32]:
prothom_alo_df_lens = prothom_alo_df['text'].str.len()

In [33]:
max(prothom_alo_df_lens)

219

In [34]:
(prothom_alo_df=='').sum()

text    1350000
dtype: int64

In [35]:
# pretraining_df_lens = pretraining_df['text'].str.len()
count = prothom_alo_df['text'].str.split().apply(len).value_counts()

In [36]:
count.index = count.index.astype(str) + ' words:'
count.sort_index(inplace=True)

In [37]:
count

text
0 words:     1350000
10 words:     144211
11 words:     136289
12 words:     125121
13 words:     114017
14 words:     100848
15 words:      87522
4 words:       39962
5 words:       75678
6 words:      106133
7 words:      130050
8 words:      143045
9 words:      147124
Name: count, dtype: int64

In [38]:
prothom_alo_dataset.reset_format()

In [39]:
prothom_alo_dataset

Dataset({
    features: ['text'],
    num_rows: 2700000
})

In [40]:
prothom_alo_dataset = prothom_alo_dataset.filter(lambda x: x["text"]!="")

Loading cached processed dataset at /home/ashahri1/.cache/huggingface/datasets/text/default-66e892579a4abe66/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2/cache-37bc92e3ecf68258.arrow


In [41]:
prothom_alo_dataset

Dataset({
    features: ['text'],
    num_rows: 1350000
})

In [42]:
prothom_alo_dataset[:20]

{'text': ['রিয়াল ভাবতে পারে তারা আসলে 1111111111 পয়েন্টে এগিয়ে ।',
  'পরে সেখানে সংক্ষিপ্ত সমাবেশ অনুষ্ঠিত হয় ।',
  'বেঁচে থাকার কথা ভাবিনি বলছিলেন নূর মোহাম্মদ ।',
  'তিনি চুয়াডাঙ্গা শহরের মল্লিকপাড়া এলাকার সাইফুল ইসলামের স্ত্রী ।',
  'তাঁকে ধুনট উপজেলা স্বাস্থ্য কমপ্লেক্সে আনা হলে কর্তব্যরত চিকিৎসক তাঁকে মৃত বলে ঘোষণা করেন ।',
  'আমরা আসলে পরীক্ষা নিয়ে ব্যস্ত হয়ে পড়েছি ।',
  'কয়েকটি বাসের ওপরে ক্যারিয়ার পাওয়া গেছে ।',
  'ফলে নজরুলগীতির অন্তর্গত মর্ম বুঝতে শুরু করি ।',
  'সেখানে হঠাৎ এত লম্বা সময় ধরে করতালির কারণটা কী ?',
  'অতিথি হিসেবে থাকবেন বাগেরহাট 1111111111 আসনের সাংসদ শেখ হেলাল উদ্দিন ।',
  'আয়োজকেরা জানালেন উৎসবে 1111111111 প্যাভিলিয়ন আর 1111111111 স্টল রয়েছে ।',
  'তাঁর সঙ্গে কাজ করারও আগ্রহ প্রকাশ করেন আমির ।',
  'রাজনীতি থেকে দূরে থাকুন ।',
  'মুহম্মদ ইকবাল হোসেন কাউন্সেলর প্রেস বাংলাদেশ হাইকমিশন ইসলামাবাদ পাকিস্তান ।',
  'আর যখন আপনি দলে আশানুরূপ অবদান রাখতে পারবেন না তখনই সময় ইতি টানার ।',
  'টেস্ট ম্যাচও এভাবে জেতা যায় !',
  'সেখানে যুক্ত হয়েছে কালিকা আমাদেরই ভাটি অঞ্চ

In [18]:
def get_training_corpus():
    for i in range(0, len(prothom_alo_dataset), 1000):
        yield prothom_alo_dataset[i : i+1000]["text"]

In [22]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

In [23]:
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

In [24]:
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFKC()]
)

In [25]:
print(tokenizer.normalizer.normalize_str('রিয়াল ভাবতে পারে তারা আসলে 1111111111 পয়েন্টে এগিয়ে ।'))

রিয়াল ভাবতে পারে তারা আসলে 1111111111 পয়েন্টে এগিয়ে ।


In [26]:
tokenizer.pre_tokenizer = pre_tokenizers.BertPreTokenizer()

In [27]:
tokenizer.pre_tokenizer.pre_tokenize_str('রিয়াল ভাবতে পারে তারা আসলে 1111111111 পয়েন্টে এগিয়ে ।')

[('রিয়াল', (0, 5)),
 ('ভাবতে', (6, 11)),
 ('পারে', (12, 16)),
 ('তারা', (17, 21)),
 ('আসলে', (22, 26)),
 ('1111111111', (27, 37)),
 ('পয়েন্টে', (38, 45)),
 ('এগিয়ে', (46, 51)),
 ('।', (52, 53))]

In [28]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(
    vocab_size=30522, special_tokens=special_tokens
)

In [29]:
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [30]:
encoding = tokenizer.encode('রিয়াল ভাবতে পারে তারা আসলে 1111111111 পয়েন্টে এগিয়ে ।')
print(encoding.tokens)

['রিয়াল', 'ভাবতে', 'পারে', 'তারা', 'আসলে', '1111111111', 'পয়েন্টে', 'এগিয়ে', '।']


In [31]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

2 3


In [32]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [33]:
encoding = tokenizer.encode('রিয়াল ভাবতে পারে তারা আসলে 1111111111 পয়েন্টে এগিয়ে ।')
print(encoding.tokens)

['[CLS]', 'রিয়াল', 'ভাবতে', 'পারে', 'তারা', 'আসলে', '1111111111', 'পয়েন্টে', 'এগিয়ে', '।', '[SEP]']


In [34]:
encoding = tokenizer.encode("এ ছাড়া শিক্ষাপ্রতিষ্ঠানেও চলবে প্রচারণা ।","সহযোগিতা করছে তথ্য ও যোগাযোগপ্রযুক্তি আইসিটি বিভাগ ।")
print(encoding.tokens)
print(encoding.type_ids)

['[CLS]', 'এ', 'ছাড়া', 'শিক্ষাপ্রতিষ্ঠানে', '##ও', 'চলবে', 'প্রচারণা', '।', '[SEP]', 'সহযোগিতা', 'করছে', 'তথ্য', 'ও', 'যোগাযোগপ্রযুক্তি', 'আইসিটি', 'বিভাগ', '।', '[SEP]']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [35]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [36]:
tokenizer.decode(encoding.ids)

'এ ছাড়া শিক্ষাপ্রতিষ্ঠানেও চলবে প্রচারণা । সহযোগিতা করছে তথ্য ও যোগাযোগপ্রযুক্তি আইসিটি বিভাগ ।'

In [37]:
tokenizer.enable_truncation(max_length=512)

In [39]:
tokenizer.save("wordpiece_tokenizer_prothom_alo.json")

In [44]:
from transformers import PreTrainedTokenizerFast

tokenizer = PreTrainedTokenizerFast(
    #tokenizer_object=tokenizer,
    tokenizer_file="wordpiece_tokenizer_prothom_alo.json", # You can load from the tokenizer file, alternatively
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
    return_special_tokens_mask = True,
    model_max_length = 512,
)

In [20]:
tokenizer.save_pretrained("wordpiece_tokenizer_prothom_alo")

('wordpiece_tokenizer_prothom_alo/tokenizer_config.json',
 'wordpiece_tokenizer_prothom_alo/special_tokens_map.json',
 'wordpiece_tokenizer_prothom_alo/tokenizer.json')

In [1]:
#test tokenizer
# tokenizer.tokenize("শিক্ষাপ্রতিষ্ঠানেও")
# tokenizer.tokenize("স্পর্শে")
# tokenizer.tokenize("পাচজনকে")
# tokenizer.tokenize("অকপটভাবে")
# tokenizer.tokenize("অকল্যাণকর")
# tokenizer.tokenize("বিনম্রতা") 
# tokenizer.tokenize("পথপ্রদর্শক")
# tokenizer.tokenize("গণপরিবহনের")
# tokenizer.tokenize("অকাজ")
tokenizer.tokenize("সুকৌশলে")

NameError: name 'tokenizer' is not defined

In [25]:
# # from transformers import BertTokenizerFast
# from transformers import PreTrainedTokenizerFast

# tokenizer = PreTrainedTokenizerFast.from_pretrained("wordpiece_tokenizer_hindi_live")

In [22]:
from transformers import BertConfig, BertForMaskedLM


# Set a configuration for our RoBERTa model
unigram_bert_config = BertConfig(pad_token_id=tokenizer.pad_token_id)

# Building the model from the config
# Model is randomly initialized
model = BertForMaskedLM(unigram_bert_config)

print(unigram_bert_config)

BertConfig {
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.16.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [23]:
text = 'পরে সেখানে সংক্ষিপ্ত সমাবেশ [MASK] হয় ।'

In [24]:
tokenizer.tokenize(text)

['পরে', 'সেখানে', 'সংক্ষিপ্ত', 'সমাবেশ', '[MASK]', 'হয়', '।']

In [25]:
import torch 

inputs = tokenizer(text, return_tensors="pt")
# inputs.to("cuda")

token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]

# # Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> পরে সেখানে সংক্ষিপ্ত সমাবেশ খেলানোর হয় ।'
'>>> পরে সেখানে সংক্ষিপ্ত সমাবেশ বাড়াবে হয় ।'
'>>> পরে সেখানে সংক্ষিপ্ত সমাবেশ টেস্টে হয় ।'
'>>> পরে সেখানে সংক্ষিপ্ত সমাবেশ পাগলা হয় ।'
'>>> পরে সেখানে সংক্ষিপ্ত সমাবেশ ডেরা হয় ।'


In [26]:
def tokenize_function(examples):
    result = tokenizer(examples["text"], padding="max_length", max_length=80, truncation=True)
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result


# Use batched=True to activate fast multithreading!
tokenized_datasets = prothom_alo_dataset.map(
    tokenize_function, batched=True, remove_columns=["text"]
)
tokenized_datasets

Loading cached processed dataset at /home/ashahri1/.cache/huggingface/datasets/text/default-66e892579a4abe66/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2/cache-90a3a3533f0fca90.arrow


Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
    num_rows: 1350000
})

In [27]:
# temp = tokenized_datasets.filter(lambda x:x if 1 in x["input_ids"] else None)

In [28]:
# temp

In [29]:
tokenized_datasets = tokenized_datasets.remove_columns("token_type_ids")

In [30]:
tokenized_datasets[0]

{'input_ids': [2,
  3574,
  4844,
  636,
  731,
  2313,
  287,
  8587,
  1425,
  62,
  3,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'word_ids': [None,
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,


In [31]:
def group_texts(examples):
    # Create a new labels column
    examples["labels"] = examples["input_ids"].copy()
    return examples

In [32]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
lm_datasets

Loading cached processed dataset at /home/ashahri1/.cache/huggingface/datasets/text/default-66e892579a4abe66/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2/cache-fbe56ba1b7233d3c.arrow


Dataset({
    features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 1350000
})

In [33]:
tokenizer.decode(lm_datasets[1]["input_ids"])

'[CLS] পরে সেখানে সংক্ষিপ্ত সমাবেশ অনুষ্ঠিত হয় । [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [34]:
tokenizer.decode(lm_datasets[1]["labels"])

'[CLS] পরে সেখানে সংক্ষিপ্ত সমাবেশ অনুষ্ঠিত হয় । [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [35]:
tokenizer.pad_token_id

1

In [36]:
import collections
import numpy as np
np.random
from transformers import default_data_collator

wwm_probability = 0.15


def bangla_data_collator(features):
    for feature in features:
#         word_ids = feature.pop("word_ids")

#         # Create a map between words and corresponding token indices
#         mapping = collections.defaultdict(list)
#         current_word_index = -1
#         current_word = None
#         for idx, word_id in enumerate(word_ids):
#             if word_id is not None:
#                 if word_id != current_word:
#                     current_word = word_id
#                     current_word_index += 1
#                 mapping[current_word_index].append(idx)

        # Randomly mask words
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        mask = np.random.binomial(1, wwm_probability, (len(input_ids),))
        special_tokens =  [tokenizer.unk_token_id, tokenizer.pad_token_id, tokenizer.cls_token_id, \
                           tokenizer.sep_token_id, tokenizer.mask_token_id]
        
        new_labels = [-100] * len(labels)
        for idx in np.where(mask)[0]:
#             word_id = word_id.item()
#             print(word_id)
#             for idx in mapping[word_id]:
#             if word_ids[idx] is not None:
            if input_ids[idx] not in special_tokens:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id
            feature["labels"] = new_labels
        
    return default_data_collator(features)

In [37]:
from transformers import DataCollatorForLanguageModeling

# data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
lm_datasets = lm_datasets.remove_columns(["word_ids"])
data_collator = bangla_data_collator

In [38]:
samples = [lm_datasets[i] for i in range(1)]
# for sample in samples:
#     _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] রিয়াল ভাবতে পারে তারা [MASK] [MASK] [MASK] এগিয়ে । [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'


In [39]:
samples = [lm_datasets[i] for i in range(1)]

chunk = data_collator(samples)
print(chunk["input_ids"])
print(chunk["labels"])

tensor([[   2, 3574, 4844,  636,  731, 2313,  287,    4,    4,   62,    3,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1]])
tensor([[-100, -100, -100, -100, -100, -100, -100, 8587, 1425, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100,

In [40]:
# train_size = 10_000
# test_size = int(0.1 * train_size)

downsampled_dataset = lm_datasets.train_test_split(
    train_size=0.8, seed=42
)
downsampled_dataset

Loading cached split indices for dataset at /home/ashahri1/.cache/huggingface/datasets/text/default-66e892579a4abe66/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2/cache-d7d8ec40ba21b3b3.arrow and /home/ashahri1/.cache/huggingface/datasets/text/default-66e892579a4abe66/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2/cache-b22cf992e3c56ac1.arrow


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1080000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 270000
    })
})

In [40]:
for idx, sample in enumerate(downsampled_dataset["train"]["input_ids"][:3]):
    print(f"'>>> Article {idx} length: {len(sample)}'")

'>>> Article 0 length: 80'
'>>> Article 1 length: 80'
'>>> Article 2 length: 80'


In [41]:
#  disable weights and biases logging
import os
os.environ["WANDB_DISABLED"] = "true"

In [42]:
from transformers import TrainingArguments

batch_size = 64
# Show the training loss with every epoch|
logging_steps = len(downsampled_dataset["train"]) // batch_size


training_args = TrainingArguments(
    num_train_epochs = 6,
    report_to = None,
    output_dir="models/wordpiece/bert-base-pretrained-prothom-alo",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    #push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,
    load_best_model_at_end=True,
    save_strategy = "epoch",
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [43]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [49]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 270000
  Batch size = 64


>>> Perplexity: 57.23


In [50]:
trainer.train()

***** Running training *****
  Num examples = 1080000
  Num Epochs = 6
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 101250


Epoch,Training Loss,Validation Loss
1,4.207700,4.134030
2,4.078600,4.023372
3,3.968100,3.936831
4,3.874200,3.879529
5,3.808500,3.826794
6,3.763100,3.811265


Saving model checkpoint to models/wordpiece/bert-base-pretrained-prothom-alo/checkpoint-16875
Configuration saved in models/wordpiece/bert-base-pretrained-prothom-alo/checkpoint-16875/config.json
Model weights saved in models/wordpiece/bert-base-pretrained-prothom-alo/checkpoint-16875/pytorch_model.bin
tokenizer config file saved in models/wordpiece/bert-base-pretrained-prothom-alo/checkpoint-16875/tokenizer_config.json
Special tokens file saved in models/wordpiece/bert-base-pretrained-prothom-alo/checkpoint-16875/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 270000
  Batch size = 64
Saving model checkpoint to models/wordpiece/bert-base-pretrained-prothom-alo/checkpoint-33750
Configuration saved in models/wordpiece/bert-base-pretrained-prothom-alo/checkpoint-33750/config.json
Model weights saved in models/wordpiece/bert-base-pretrained-prothom-alo/checkpoint-33750/pytorch_model.bin
tokenizer config file saved in models/wordpiece/bert-base-pretrained-prothom-al

TrainOutput(global_step=101250, training_loss=3.9500376157407406, metrics={'train_runtime': 19335.866, 'train_samples_per_second': 335.129, 'train_steps_per_second': 5.236, 'total_flos': 2.6649487296e+17, 'train_loss': 3.9500376157407406, 'epoch': 6.0})

In [51]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 270000
  Batch size = 64


>>> Perplexity: 44.82


In [52]:
trainer.save_model()

Saving model checkpoint to models/wordpiece/bert-base-pretrained-prothom-alo
Configuration saved in models/wordpiece/bert-base-pretrained-prothom-alo/config.json
Model weights saved in models/wordpiece/bert-base-pretrained-prothom-alo/pytorch_model.bin
tokenizer config file saved in models/wordpiece/bert-base-pretrained-prothom-alo/tokenizer_config.json
Special tokens file saved in models/wordpiece/bert-base-pretrained-prothom-alo/special_tokens_map.json


In [56]:
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

eval/loss,▅█▆▄▃▁▁▁
eval/runtime,█▅▅▅▅▄▅▁
eval/samples_per_second,▁▄▄▄▄▄▄█
eval/steps_per_second,▁▄▄▄▄▄▄█
train/epoch,▁▁▂▂▄▄▅▅▇▇████
train/global_step,▁▂▂▃▃▄▅▆▆▇▇████
train/learning_rate,█▇▅▄▂▁
train/loss,▇▁▁▂▄█
train/total_flos,▁
train/train_loss,▁
train/train_runtime,▁


In [44]:
model = BertForMaskedLM.from_pretrained("models/wordpiece/bert-base-pretrained-prothom-alo")

loading configuration file models/wordpiece/bert-base-pretrained-prothom-alo/config.json
Model config BertConfig {
  "_name_or_path": "models/wordpiece/bert-base-pretrained-prothom-alo",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.16.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file models/wordpiece/bert-base-pretrained-prothom-alo/pytorch_model.bin
All model checkpoint weights were used when initializing BertForMaskedLM.

All the weights of BertForMaskedLM were initialized from

In [45]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("models/wordpiece/bert-base-pretrained-prothom-alo")

Didn't find file models/wordpiece/bert-base-pretrained-prothom-alo/added_tokens.json. We won't load it.
loading file None
loading file models/wordpiece/bert-base-pretrained-prothom-alo/special_tokens_map.json
loading file models/wordpiece/bert-base-pretrained-prothom-alo/tokenizer_config.json
loading file models/wordpiece/bert-base-pretrained-prothom-alo/tokenizer.json


In [85]:
def insert_random_mask(batch):
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    masked_inputs = data_collator(features)
    # Create a new "masked" column for each column in the dataset
    return {"masked_" + k: v.numpy() for k, v in masked_inputs.items()}

In [87]:
# downsampled_dataset = downsampled_dataset.remove_columns(["word_ids"])
eval_dataset = downsampled_dataset["test"].map(
    insert_random_mask,
    batched=True,
    remove_columns=downsampled_dataset["test"].column_names,
)
eval_dataset = eval_dataset.rename_columns(
    {
        "masked_input_ids": "input_ids",
        "masked_attention_mask": "attention_mask",
        "masked_labels": "labels",
        'masked_token_type_ids' : 'token_type_ids'
    }
)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [88]:
eval_dataset[0]

{'input_ids': [2,
  4320,
  468,
  4,
  9912,
  31,
  3,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

In [90]:
samples = [eval_dataset[i] for i in range(10)]

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] আসল নাম [MASK] রায়হান । [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

'>>> [CLS] তবে [MASK] [MASK] [MASK] সমঝোতা না হওয়ায় ওই [MASK] কোনো সিদ্ধান্ত ছাড়াই শেষ [MASK] [MASK] [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

In [95]:
for idx, _ in enumerate(eval_dataset['input_ids']):
    indexes = [i for i, x in enumerate(eval_dataset[idx]['input_ids']) if x == 4]
    references = [i for i, x in enumerate(eval_dataset[idx]['labels']) if x != -100]
    if indexes != references:
        print(idx)

In [76]:
from torch.utils.data import DataLoader
from transformers import default_data_collator

batch_size = 4
train_dataloader = DataLoader(
    downsampled_dataset["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)
eval_dataloader = DataLoader(
    eval_dataset, batch_size=batch_size, collate_fn=default_data_collator
)

In [77]:
downsampled_dataset["train"]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 10000
})

In [78]:
downsampled_dataset["test"]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 1000
})

In [141]:
# from transformers import BertConfig, BertForMaskedLM

# # Building the config
# config = BertConfig()

# # Building the model from the config
# # Model is randomly initialized
# model = BertForMaskedLM(config)

In [44]:
tokenizer = PreTrainedTokenizerFast.from_pretrained("models/wordpiece/bert-base-pretrained-bdnews24")

In [45]:
model = BertForMaskedLM.from_pretrained("models/wordpiece/bert-base-pretrained-bdnews24")

In [83]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [84]:
# from accelerate import Accelerator

# accelerator = Accelerator()
# model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
#     model, optimizer, train_dataloader, eval_dataloader
# )

In [85]:
from transformers import get_scheduler

num_train_epochs = 1
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [87]:
output_dir = "models/wordpiece/bert-base-pretrained-bdnews24-static"

In [88]:
from tqdm.auto import tqdm
import torch
import math

progress_bar = tqdm(range(num_training_steps))

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        #accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    losses = []
    for step, batch in enumerate(eval_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        loss = outputs.loss
        #losses.append(accelerator.gather(loss.repeat(batch_size)))
        losses.append(loss.repeat(batch_size))

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    print(f">>> Epoch {epoch}: Loss: {torch.mean(losses)}")
    print(f">>> Epoch {epoch}: Perplexity: {perplexity}")

    # Save and upload
    #accelerator.wait_for_everyone()
    #unwrapped_model = accelerator.unwrap_model(model)
    #unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    #if accelerator.is_main_process:
        #tokenizer.save_pretrained(output_dir)

  0%|          | 0/2500 [00:00<?, ?it/s]

Configuration saved in models/wordpiece/bert-base-pretrained-bdnews24-static\config.json


>>> Epoch 0: Loss: nan
>>> Epoch 0: Perplexity: nan


Model weights saved in models/wordpiece/bert-base-pretrained-bdnews24-static\pytorch_model.bin
tokenizer config file saved in models/wordpiece/bert-base-pretrained-bdnews24-static\tokenizer_config.json
Special tokens file saved in models/wordpiece/bert-base-pretrained-bdnews24-static\special_tokens_map.json


In [94]:
import torch
torch.cuda.empty_cache()